# Análise do Catálogo da Netflix

Alexandre Souza - 12559506

Eduardo Zaffari Monteiro - 12559490

Lucas Ivars Cadima Ciziks - 12559472

## Introdução
Em 2021, a indústria de entretenimento audiovisual provou que conseguiu superar a pandemia do COVID-19 no que tange ao seu faturamento total. Segundo o relatório anual da Motion Picture Association (MPA), no ano passado foram arrecadados US$ 99,7 bilhões, número que supera os US$ 98,1 bilhões de 2019, ano anterior à crise sanitária. 

Da arrecadação total, 72% refere-se ao consumo de mídia digital, o equivalente a US$ 71,9 bilhões, impulsionado pelos serviços de mídia digital (serviços de *streaming* e *TV Everywhere*), que contavam com mais de 1,3 bilhão de assinaturas ativas no último ano, das quais 221,6 milhões eram da Netflix.

Em se tratando da Netflix, que hoje possui 45,2% da fatia de mercado de serviços de *streaming*, aproximadamente 17,8% de suas assinaturas são da América Latina, contra os 33,2% da EMEA, região que corresponde a Europa, Oriente Médio e África. Contudo, ainda segundo o relatório da MPA, o mercado de serviços digitais aumentou 29% nos países latino-americanos em 2021 (quando comparado a 2020), em contraste com os  21% da região comercial EMEA.

Tendo isso em vista, o objetivo deste relatório é tentar provar, por meio da análise exploratória dos dados e teste de hipóteses, que houve maior investimento em produções de origem europeia em detrimento das de origem latino-americana, explicitando a desigualdade cultural entre as duas regiões.

## Desenvolvimento

### Coleta e Limpeza dos dados

In [ ]:
# Realizando importação de Bibliotecas
import pandas as pd
import numpy as np
import json
from scipy import stats

: 

In [65]:
# Carregando Dados referentes ao Catálogo da Netflix
netflix_raw = pd.read_csv('https://raw.githubusercontent.com/Cizika/netflix-analysis/main/netflix_titles.csv', index_col=0)
netflix_raw.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [66]:
# Carregando Dados de países/continentes
with open('countries.json') as json_file:
    countries = json.load(json_file)

In [67]:
# Lista de países europeus
european_list = countries["Europe"]["countries"]
european_list

['Albania',
 'Andorra',
 'Ã…land',
 'Austria',
 'Belarus',
 'Belgium',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Faroe Islands',
 'Finland',
 'France',
 'Germany',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Guernsey',
 'Hungary',
 'Iceland',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Jersey',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Macedonia',
 'Malta',
 'Moldova',
 'Monaco',
 'Montenegro',
 'Netherlands',
 'Norway',
 'Poland',
 'Romania',
 'Russia',
 'Serbia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Slovakia',
 'Slovenia',
 'Svalbard and Jan Mayen',
 'San Marino',
 'Ukraine',
 'United Kingdom']

In [68]:
# Lista de países latinos
latin_list = countries["Latin America"]["countries"]
latin_list

['Costa Rica',
 'Cuba',
 'Haiti',
 'Panama',
 'Saint BartÃ©lemy',
 'El Salvador',
 'Saint Martin',
 'Argentina',
 'Belize',
 'Bolivia',
 'Brazil',
 'Chile',
 'Colombia',
 'Ecuador',
 'Dominican Republic',
 'Guatemala',
 'Honduras',
 'Mexico',
 'Nicaragua',
 'Paraguay',
 'Peru',
 'Puerto Rico',
 'Uruguay',
 'Venezuela']

In [1]:
# Excluindo NAs
netflix_df = netflix_raw[["country","date_added"]].dropna()

# Coletando somente o ano de adição dos filmes ao catálogo
netflix_df["year_added"] = netflix_df["date_added"].apply(lambda x: int(x.split(", ")[1]))

netflix_df

NameError: name 'netflix_raw' is not defined

In [114]:
# Coletando amostra dos filmes adicionados em 2021
amostra_raw = netflix_df[netflix_df["year_added"] == 2021].copy()

# Individualizando país por linha
amostra_raw = amostra_raw.assign(country=amostra_raw["country"].str.split(", ")).explode("country")

amostra_raw

,country,date_added,year_added
show_id,,,
s1,United States,"September 25, 2021",2021
s2,South Africa,"September 24, 2021",2021
s5,India,"September 24, 2021",2021
s8,United States,"September 24, 2021",2021
s8,Ghana,"September 24, 2021",2021
...,...,...,...
s8321,Sweden,"January 5, 2021",2021
s8321,Norway,"January 5, 2021",2021
s8435,United States,"January 1, 2021",2021


In [115]:
# Calculando Número de Filmes por País
amostra = amostra_raw.groupby("country").size().to_frame()
amostra.drop("", inplace=True)
amostra.reset_index(inplace=True)
amostra.rename(columns={"country":"País", 0: "Número de Filmes" }, inplace=True)

amostra

,País,Número de Filmes
0,Algeria,1
1,Argentina,12
2,Australia,24
3,Austria,1
4,Bangladesh,1
...,...,...
67,"United Kingdom,",1
68,United States,627
69,Uruguay,2
70,Venezuela,1


In [116]:
# Dicotomizando por Variável Qualitativa is_latin e is_european
amostra["Latino"] = amostra["País"].isin(latin_list)
amostra["Europeu"] = amostra["País"].isin(european_list)

amostra

,País,Número de Filmes,Latino,Europeu
0,Algeria,1,False,False
1,Argentina,12,True,False
2,Australia,24,False,False
3,Austria,1,False,True
4,Bangladesh,1,False,False
...,...,...,...,...
67,"United Kingdom,",1,False,False
68,United States,627,False,False
69,Uruguay,2,True,False
70,Venezuela,1,True,False


### Teste de Normalidade

In [ ]:
shapiro_test = stats.shapiro(x)
shapiro_test

### Teste para Comparação de Duas Populações Independentes

In [123]:
# Calculando número total de filmes Latinos e Europeus
total_filmes_latinos = amostra[amostra["Latino"] == True]["Número de Filmes"].sum()
total_filmes_europeus = amostra[amostra["Europeu"] == True]["Número de Filmes"].sum()

print(f"Em 2021, foram adicionados {total_filmes_latinos} filmes latinos e {total_filmes_europeus} filmes europeus no catálogo da Netflix.")

Em 2021, foram adicionados 58 filmes latinos e 337 filmes europeus no catálogo da Netflix.


In [126]:
# Calculando número total de países latinos e europeus
n_latinos = len(latin_list)
n_europeus = len(european_list)

print(f"A América Latina é formada por {n_latinos} países. Enquanto o continente Europeu possui {n_europeus}.")

A América Latina é formada por 24 países. Enquanto o continente Europeu possui 52.
